切换transformers版本 pip install transformers==4.30.2

In [ ]:
# from modelscope.utils.hf_util import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch
import json
from tqdm import tqdm
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("/mnt/workspace/.cache/modelscope/hub/ZhipuAI/chatglm3-6b",
                                          trust_remote_code=True)
device = torch.device('cuda:0')  # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained("/mnt/workspace/.cache/modelscope/hub/ZhipuAI/chatglm3-6b",
                                             device_map={"": device},
                                             torch_dtype="auto",
                                             trust_remote_code=True)
model.to(device)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    # do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40
)
hf = HuggingFacePipeline(pipeline=pipe)

### Chatglm3+KG-output

In [ ]:
# 读取 JSON 文件
file_path = 'aikps_output.json'  # 替换为你的文件路径

# 打开并读取 JSON 文件
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 初始化一个空列表，用于存储结果
results = []

# 使用 tqdm 创建进度条
for item in tqdm(data, desc="处理进度"):
    question = item[0]
    context = item[1][0]
    template ="""### 这是一道小学三年级人工智能学科的问题，请使用以下上下文来增强你的回答。
        {context}
        ### 问题: {question}
        ### 以{{"答案": ""}}的JSON格式返回最终结果。
    """
    prompt = PromptTemplate.from_template(template)
    # 假设 hf 是已经定义好的处理函数
    chain = prompt | hf
    # 修改传递给chain.invoke的参数，确保变量名和模板中的一致
    result = chain.invoke({"context": context, "question": question})

    # 将 context、question 和答案添加到结果列表中
    results.append([context, question, result])

# 将结果列表转换为 DataFrame
df = pd.DataFrame(results, columns=['Context', 'Question', 'Answer'])

# 将 DataFrame 写入新的 Excel 文件
excel_path = 'Chatglm3+KG-output.xlsx'  # 替换为你想要保存的 Excel 文件路径
df.to_excel(excel_path, index=False)

In [ ]:
question = "人工智能科学家有谁？"
context = "'3.长得像人类的机器人才叫人工智能。(______)'"
template ="""### 这是一道小学三年级人工智能学科的问题，请使用以下上下文来增强你的回答。
    {context}
    ### 问题: {question}
    ### 以{{"答案": ""}}的JSON格式返回最终结果。
"""
prompt = PromptTemplate.from_template(template)
# 假设 hf 是已经定义好的处理函数
chain = prompt | hf
# 修改传递给chain.invoke的参数，确保变量名和模板中的一致
result = chain.invoke({"context": context, "question": question})
print(result)

### Chatglm3-output

In [ ]:
# 读取 JSON 文件
file_path = 'aikps_output.json'  # 替换为你的文件路径

# 打开并读取 JSON 文件
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 初始化一个空列表，用于存储结果
results = []

# 使用 tqdm 创建进度条
for item in tqdm(data, desc="处理进度"):
    question = item[0]
    template ="""### 这是一道小学三年级人工智能学科的问题，请给出你的答案。
        ### 问题: {question}
        ### 以{{"答案": ""}}的JSON格式返回最终结果。
    """
    prompt = PromptTemplate.from_template(template)
    # 假设 hf 是已经定义好的处理函数
    chain = prompt | hf
    # 修改传递给chain.invoke的参数，确保变量名和模板中的一致
    result = chain.invoke({"question": question})

    # 将 context、question 和答案添加到结果列表中
    results.append([question, result])

# 将结果列表转换为 DataFrame
df = pd.DataFrame(results, columns=['Question', 'Answer'])

# 将 DataFrame 写入新的 Excel 文件
excel_path = 'Chatglm3-output.xlsx'  # 替换为你想要保存的 Excel 文件路径
df.to_excel(excel_path, index=False)

In [ ]:
question = "人工智能科学家有谁？"
template ="""### 这是一道小学三年级人工智能学科的问题，请给出你的答案。
    ### 问题: {question}
    ### 以{{"答案": ""}}的JSON格式返回最终结果。
"""
prompt = PromptTemplate.from_template(template)
# 假设 hf 是已经定义好的处理函数
chain = prompt | hf
# 修改传递给chain.invoke的参数，确保变量名和模板中的一致
result = chain.invoke({"question": question})
print(result)